learning rate is a step with which we change our parameters. if we decrease learning step, our parameters will change a little and very slow. if we increase learning rate than our parameters will change faster, but big learning rate is bad

we use regularization to avoid overfitting or underfitting of our model. if we increase regularization parameter too much we have a risk to underfit our data, if we decrease our regularization too much we have a risk to overfit data

error is how much our predictions differs from real rating. big error means that our model is very bad, little error means our model is good

In [4]:
lr = [0.1, 0.001, 0.0001, 0.5, 0.05, 0.0005, 0.007, 0.002, 0.0003, 0.003]
reg = [0.1, 0.2, 0.3, 0.4, 0.02, 0.05, 0.07, 0.01, 0.06, 0.08]

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
train = pd.read_csv('ua.base', sep="\t", header=None)
train.columns = ['user_id', 'item_id', 'rating', 'timestamp']
train = train.drop(['timestamp'], axis=1)
train.user_id = train.user_id - 1
train.item_id = train.item_id - 1


test = pd.read_csv('ua.test', sep="\t", header=None)
test.columns = ['user_id', 'item_id', 'rating', 'timestamp']
test = test.drop(['timestamp'], axis=1)
test.user_id = test.user_id - 1
test.item_id = test.item_id - 1

In [2]:
rng = np.random.RandomState(42)
global_mean = train.rating.mean()
u_count = train.user_id.max() + 1
i_count = train.item_id.max() + 1
n_factors = 10
u_factors = rng.normal(0, 0.1, (u_count, n_factors))
i_factors = rng.normal(0, 0.1, (i_count, n_factors))
u_bias = np.zeros(shape=(u_count,))
i_bias = np.zeros(shape=(i_count,))
trainset = train.to_numpy()
testset = test.to_numpy()

In [5]:
def estimate(user_id, item_id):
    est = global_mean + u_bias[user_id] + i_bias[item_id]
    est += sum(u_factors[user_id] * i_factors[item_id])
    return est

def rmse(y_true, y_pred):
    return np.sqrt(((y_pred - y_true) **2).mean())

In [6]:
def predict(user_ids, item_ids):
    pred = []
    for u, i in zip(user_ids, item_ids):
        pred.append(estimate(u, i))
    return pred

In [7]:
params = []
for l in lr:
    for r in reg:
        params.append((l, r))

In [21]:
results = pd.DataFrame(columns=['lerning_rate', 'regularization', 'result'])
for u, i, r in testset[:10]:
    for parameter in params:
        lr = parameter[0]
        reg = parameter[1]
        pred = estimate(u, i)
        e = r - pred
        u_bias[u] = u_bias[u] + lr * (e - reg * u_bias[u])
        i_bias[i] = i_bias[i] + lr * (e - reg * i_bias[i])
        u_factors[u] = u_factors[u] + lr * (e * i_factors[i] - reg * u_factors[u])
        i_factors[i] = i_factors[i] + lr * (e * u_factors[u] - reg * i_factors[i])
        results = results.append(
            {
                'learning_rate' : lr,
                'regularization' : reg,
                'result' : rmse(test['rating'], predict(test['user_id'], test['item_id']))
        
            }, ignore_index=True        
        )